In [ ]:
base_path = "/content/drive/MyDrive/UFSC/INE5448"

# Dependências

In [ ]:
!pip install fastapi uvicorn pyngrok unsloth google-cloud-vision nest_asyncio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.6/64.6 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 359.3/359.3 kB 38.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 529.1/529.1 kB 47.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 38.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 45.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.6/289.6 kB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.9/122.9 MB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 55.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 140.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.9/224.9 kB 20.6 MB/s eta 0:00:00
  Attempting uninstall: torchao
    

# Carregamento do modelo LLM

Carregando LLM

In [ ]:
from unsloth import FastLanguageModel

LLAMA_DIR = base_path + "/outputs_llama3_ocr"
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = LLAMA_DIR + "/checkpoint-180", # Pasta onde salvamos
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
)
FastLanguageModel.for_inference(model) # Otimiza para gerar texto


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.11.6: Fast Llama patching. Transformers: 4.57.2.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/220 [00:00<?, ?B/s]

unsloth/llama-3-8b-Instruct-bnb-4bit does not have a padding token! Will use pad_token = <|reserved_special_token_250|>.


Unsloth 2025.11.6 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096, padding_idx=128255)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
       

# Server Loop

In [ ]:
import nest_asyncio
import uvicorn
import os
import io
import json
from fastapi import FastAPI, UploadFile, File, Form
from fastapi.responses import JSONResponse
from pyngrok import ngrok
from google.colab import userdata
from google.cloud import vision
import torch

# --- 1. CONFIGURAÇÃO GOOGLE VISION ---
try:
    # Recupera credenciais e salva arquivo temporário
    google_json = userdata.get('GOOGLE_JSON_KEY')
    with open("service_account.json", "w") as f:
        f.write(google_json)
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "service_account.json"
    vision_client = vision.ImageAnnotatorClient()
    print("✅ Google Vision configurado.")
except Exception as e:
    print(f"❌ Erro no Google Vision: {e}")

# --- 2. CONFIGURAÇÃO SERVIDOR ---
ngrok.kill()
nest_asyncio.apply()
app = FastAPI()

# System Prompt idêntico ao do treinamento
SYSTEM_PROMPT = (
    "Você é um assistente especializado em pós-processamento de OCR para documentos manuscritos históricos. "
    "Sua tarefa é corrigir erros ortográficos e de leitura gerados pelo OCR, baseando-se no contexto. "
    "Mantenha nomes próprios, datas e locais exatamente como no original. "
    "Não invente informações que não estão no texto. "
    "Se o texto estiver cortado ou incompleto, corrija apenas o que é visível."
)

@app.post("/transcrever")
async def transcrever(
    image: UploadFile = File(...),
    task_id: str = Form(...)
):
    print(f"\n{task_id:<15} | 📥 Tarefa Recebida!")

    # 1. OCR via Google Vision
    try:
        contents = await image.read()
        image_vision = vision.Image(content=contents)

        # Usa document_text_detection (melhor para manuscritos)
        response = vision_client.document_text_detection(image=image_vision)

        if response.full_text_annotation:
            ocr_text = response.full_text_annotation.text
        else:
            ocr_text = ""

        # Limpeza básica de quebras de linha excessivas para facilitar o prompt
        # (Opcional, mas ajuda a LLM a focar no texto corrido)
        ocr_text_clean = ocr_text # Mantemos original ou .replace('\n', ' ') se preferir

    except Exception as e:
        print(f"{task_id:<15} | ❌ Erro OCR: {str(e)}")
        return JSONResponse({"error": "Falha no OCR"}, status_code=500)

    print(f"{task_id:<15} | OCR Concluído: {ocr_text[:30]}...")


    # 2. Correção via LLM (Llama 3)
    print(f"{task_id:<15} | 🤖 Iniciando LLM...")

    # Montagem do Prompt no Padrão Llama 3 (Crucial para funcionar bem!)
    prompt = f"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n{SYSTEM_PROMPT}<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nCorrija este texto OCR:\n{ocr_text_clean}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"

    inputs = tokenizer([prompt], return_tensors="pt").to("cuda")

    # Geração com os parâmetros otimizados na inferência
    outputs = model.generate(
        **inputs,
        max_new_tokens=512,
        use_cache=True,
        temperature=0.4,       # Criatividade controlada
        repetition_penalty=1.1, # Evitar loops
        do_sample=True,
        top_p=0.9,
    )

    # Decodificação e Limpeza
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Lógica de extração: Pegar tudo depois de "assistant"
    try:
        # O tokenizer.decode remove as tags especiais, então sobra algo como:
        # "system\n\nPrompt...\n\nuser\n\nOCR...\n\nassistant\n\nTexto Corrigido"
        prediction = generated_text.split("assistant\n\n")[-1].strip()
    except:
        prediction = generated_text

    print(f"{task_id:<15} | ✅ Correção Finalizada!")
    print(f"{'-'*40}")

    return JSONResponse({
        "task_id": task_id,
        "ocr": ocr_text,
        "llm": prediction
    })

# --- 3. INICIALIZAÇÃO ---
try:
    ngrok.set_auth_token(userdata.get('NGROK_TOKEN'))
except:
    print("⚠️ Aviso: Token NGROK não encontrado.")

tunnel = ngrok.connect(8000)
public_url = tunnel.public_url
print(f"🚀 SERVIDOR NO AR! URL: {public_url}")
print(f"📡 Endpoint: {public_url}/transcrever")

config = uvicorn.Config(app, host="0.0.0.0", port=8000, log_level="error") # Log level error para limpar o output
server = uvicorn.Server(config)
await server.serve()

✅ Google Vision configurado.
🚀 SERVIDOR NO AR! URL: https://kaylin-wearproof-backhandedly.ngrok-free.dev
📡 Endpoint: https://kaylin-wearproof-backhandedly.ngrok-free.dev/transcrever

56743e12-fa45-4cd3-a378-991da5bf2564 | 📥 Tarefa Recebida!
56743e12-fa45-4cd3-a378-991da5bf2564 | OCR Concluído: ra counter levrei sete termo
o...
56743e12-fa45-4cd3-a378-991da5bf2564 | 🤖 Iniciando LLM...
56743e12-fa45-4cd3-a378-991da5bf2564 | ✅ Correção Finalizada!
----------------------------------------

8c2dd608-8994-4e8a-8cf2-9104511f3cc4 | 📥 Tarefa Recebida!
8c2dd608-8994-4e8a-8cf2-9104511f3cc4 | OCR Concluído: morena,
do sexo
Mabel ben o
na...
8c2dd608-8994-4e8a-8cf2-9104511f3cc4 | 🤖 Iniciando LLM...
8c2dd608-8994-4e8a-8cf2-9104511f3cc4 | ✅ Correção Finalizada!
----------------------------------------

6e762b7a-889f-4de0-ab3f-44b70577e27e | 📥 Tarefa Recebida!
6e762b7a-889f-4de0-ab3f-44b70577e27e | OCR Concluído: tural de Par Jedro
do Itararé,...
6e762b7a-889f-4de0-ab3f-44b70577e27e | 🤖 Iniciando LLM.

# Server Loop 2

In [ ]:
from google.colab import userdata
from google.cloud import vision
import os
import torch

# --- CONFIGURAÇÃO GOOGLE VISION ---
# Recupera o JSON das credenciais salvo nos Secrets do Colab
try:
    google_json = userdata.get('GOOGLE_JSON_KEY')
    # Salva temporariamente o JSON num arquivo para a lib do Google ler
    with open("service_account.json", "w") as f:
        f.write(google_json)
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "service_account.json"
    print("✅ Credenciais do Google Cloud configuradas.")
except Exception as e:
    print(f"❌ Erro ao configurar credenciais do Google: {e}")
    print("Certifique-se de adicionar 'GOOGLE_JSON_KEY' nos Secrets do Colab.")

# Inicializa o cliente do Vision
vision_client = vision.ImageAnnotatorClient()

✅ Credenciais do Google Cloud configuradas.


In [ ]:
import nest_asyncio
import uvicorn
import os
import io
import json
from fastapi import FastAPI, UploadFile, File, Form
from fastapi.responses import JSONResponse
from pyngrok import ngrok
from google.colab import userdata
from google.cloud import vision
import torch


# --- FIM CONFIGURAÇÃO GOOGLE VISION ---

ngrok.kill()
nest_asyncio.apply()

app = FastAPI()

@app.post("/transcrever")
async def transcrever(
    image: UploadFile = File(...),
    task_id: str = Form(...)
):
    print(f"{task_id:<15} | Tarefa Recebida!")

    # Google Vision aceita bytes diretos, não precisa decodificar com cv2
    contents = await image.read()

    print(f"{task_id:<15} | Enviando imagem para Google Cloud Vision API...")

    try:
        # Cria o objeto de imagem do Google Vision
        vision_image = vision.Image(content=contents)

        # Chama a API de detecção de texto (DOCUMENT_TEXT_DETECTION é melhor para manuscritos/densos)
        response = vision_client.document_text_detection(image=vision_image)
        # Ou use text_detection para textos esparsos:
        # response = vision_client.text_detection(image=vision_image)

        ocr_text = ""
        if response.text_annotations:
            # O primeiro elemento contém o texto completo
            ocr_text = response.text_annotations[0].description
        else:
            ocr_text = ""

        if response.error.message:
            raise Exception(f'{response.error.message}')

    except Exception as e:
        print(f"{task_id:<15} | ❌ Erro no Google Vision: {str(e)}")
        return JSONResponse({"task_id": task_id, "error": str(e)}, status_code=500)

    # Remove quebras de linha excessivas se necessário
    ocr_text = ocr_text.replace("\n", " ")
    print(f"{task_id:<15} | Extração finalizada: {ocr_text[:20]}...{ocr_text[-20:]}")

    max_len_estimado = int(len(ocr_text) * 1.5) + 50

    print(f"{task_id:<15} | Iniciando correções via LLM!")

    # --- INICIO BLOCO LLM (Assume que model/tokenizer já estão carregados globalmente) ---
    prompt = PROMPT_TEMPLATE.format(ocr_text)
    inputs = tokenizer([prompt], return_tensors="pt").to("cuda")

    outputs = model.generate(
        **inputs,
        max_new_tokens=min(max_len_estimado, 4096),
        use_cache=True,
        temperature=0.01,
        do_sample=True,
        repetition_penalty=1.2,
        no_repeat_ngram_size=4,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id,
    )

    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    try:
        # Lógica de parsing do output
        prediction = generated_text.split("### CORRECTION:\n")[-1].strip()
        if "### OCR TEXT" in prediction:
            prediction = prediction.split("### OCR TEXT")[0]
        prediction = prediction.strip()
    except:
        prediction = generated_text
    # --- FIM BLOCO LLM ---

    print(f"{task_id:<15} | Correção finalizada: {prediction[:20]}...{prediction[-20:]}")

    return JSONResponse({
        "task_id": task_id,
        "ocr": ocr_text[:4096],
        "llm": prediction[:min(max_len_estimado, 4096)]
    })

# Configuração do Ngrok
try:
    ngrok.set_auth_token(userdata.get('NGROK_TOKEN'))
except:
    print("Aviso: Token NGROK não encontrado em userdata.")

tunnel = ngrok.connect(8000)
public_url = tunnel.public_url
print(f"✅ SERVIDOR RODANDO! URL PÚBLICA: {public_url}")
print(f"➡️  Atualize seu endpoint.txt com: {public_url}/transcrever")

config = uvicorn.Config(app, host="0.0.0.0", port=8000, log_level="info")
server = uvicorn.Server(config)
await server.serve()

✅ SERVIDOR RODANDO! URL PÚBLICA: https://kaylin-wearproof-backhandedly.ngrok-free.dev
➡️  Atualize seu endpoint.txt com: https://kaylin-wearproof-backhandedly.ngrok-free.dev/transcrever


INFO:     Started server process [3110]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


f6a73df9-ce09-49e6-ae87-fc24f207879f | Tarefa Recebida!
f6a73df9-ce09-49e6-ae87-fc24f207879f | Enviando imagem para Google Cloud Vision API...
f6a73df9-ce09-49e6-ae87-fc24f207879f | Extração finalizada: Canarição, githa leg...gar kaipon Seven 108
f6a73df9-ce09-49e6-ae87-fc24f207879f | Iniciando correções via LLM!
f6a73df9-ce09-49e6-ae87-fc24f207879f | Correção finalizada: Candido, filho legit... é Correa da Camera.
INFO:     179.0.58.194:0 - "POST /transcrever HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [3110]
